In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import json
import sys
from pathlib import Path
import re 

import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blp import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.pricing.cds import get_isda_cds, get_isda_cds_specs
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [3]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

#### Local data

In [4]:
curve_hist = pd.read_csv(data_path / 'curve_history.csv', parse_dates=['date'], dayfirst=True)
cds_ref = pd.read_csv(data_path / 'cds_reference.csv', parse_dates=['maturity'], dayfirst=True)
cds_hist = pd.read_csv(data_path / 'cds_history.csv', parse_dates=['date'], dayfirst=True)

#### Settings

In [5]:
cds_name = "TURKEY CDS USD SR 5Y D14 Curncy"
curve_name =  "USD ISDA Standard Rates Swap Curve" # "EUR ISDA Standard Rates Swap Curve"  # 'USD ISDA CDS SOFR Rates Swap Curve'
trade_date = pd.Timestamp.today().floor('d')

date_start = pd.Timestamp('2021-06-02')
date_end = pd.Timestamp('2022-06-23')

dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [9]:
bq = BlpQuery(timeout=50000).start()

In [6]:
securities = [
    "TURKEY CDS USD SR 5Y D14 Curncy"     
]

fields = [
    'security_des',
    'issuer_bulk',
    'crncy',
    'cds_recovery_rt',
    'tenor',
    'px_last',
    # '5y_mid_cds_spread',
]

In [7]:
curve_name = 'USD ISDA CDS SOFR Rates Swap Curve'
curve = get_curve(curve_name, curves)

In [10]:
cds_spreads = bq.bdp(
    securities,
    fields=fields
).rename({'crncy': 'currency', 'cds_recovery_rt': 'recovery'}, axis=1)

In [11]:
curve_rates = bq.bdp(
    securities=[memb['ticker'] for memb in curve['members']],
    fields=['px_last']
)# .assign(tenor=[memb['tenor'] for memb in curve['members']])

In [12]:
curve_rates['tenor'] = curve_rates['security'].replace({memb['ticker']: memb['tenor'] for memb in curve['members']})

In [13]:
# cds_specs = get_cds_specs(cds_name, cds_ref)
# cds_curve = get_cds_curve(cds_name, cds_ref, cds_hist)
# cds_spread  = get_cds_spreads(cds_name, cds_hist).loc[date_start:date_end]
risk_free_curve = curve_rates.set_index('tenor')['px_last']
notional = 10000000
coupon=100
flat = True
n =10

In [14]:
data = pd.read_clipboard()

In [17]:
res = []
for index, cds_specs in data.iterrows():
    spread = cds_specs['spread']
    coupon = cds_specs['coupon']
    recovery = cds_specs['recovery']
    upfront = cds_specs['upfront']
    cds = get_isda_cds(
        risk_free_curve=risk_free_curve,
        trade_date=trade_date,
        cds_spread=spread,
        recovery=recovery,
        upfront=72,
        notional=notional,
        coupon=coupon,
        rebates_accrual=True
    )
    
    res.append({'npv': cds.NPV(), 'cds_implied_price': 100 * (1+cds.fairUpfront()), 'spread': spread})

In [18]:
res

[]

In [25]:
spread

179.0

In [19]:
res = []
for index, cds_specs in cds_spreads.iterrows():
    spread = cds_specs['px_last']
    cds = get_isda_cds_specs(
        cds_specs=cds_specs,
        risk_free_curve=risk_free_curve,
        trade_date=trade_date,
        cds_spread=spread,
        notional=notional,
        coupon=coupon,
        rebates_accrual=True
    )
    
    res.append({'npv': cds.NPV(), 'cds_implied_price': 100 * (1+cds.fairUpfront()), 'spread': spread, 'upfront': cds.fairUpfront()})

In [20]:
res

[{'npv': 2624427.7255263403,
  'cds_implied_price': 73.74990934143399,
  'spread': 827.0,
  'upfront': -0.26250090658566005}]

In [ ]:
res = pd.DataFrame(res).assign(
    issuer=cds_spreads['issuer_bulk'],
    des=cds_spreads['security_des'],
    wgt=wgt
)

In [ ]:
res

In [ ]:
res = []
for n, (index, cds_specs) in zip(wgt, cds_spreads.iterrows()):
    spread = cds_specs['px_last']
    cds = get_isda_cds(
        cds_specs=cds_specs,
        risk_free_curve=risk_free_curve,
        trade_date=trade_date,
        cds_spread=spread,
        notional=n,
        coupon=coupon,
        rebates_accrual=True
    )
    
    res.append({'npv': cds.NPV(), 'cds_implied_price': 100 * (1+cds.fairUpfront()), 'spread': spread})

In [ ]:
res = pd.DataFrame(res, index=cds_spreads['issuer_bulk']).assign(wgt=wgt)

In [ ]:
res['npv'].sum()

In [ ]:
cds_spreads

In [ ]:
((res['wgt'] * res['spread']) / res['wgt'].sum()).sum()

In [ ]:
cds = get_isda_cds(
    cds_specs=cds_specs,
    risk_free_curve=risk_free_curve,
    trade_date=trade_date,
    cds_spread=cds_specs['px_last'],
    notional=notional,
    coupon=coupon
)

In [ ]:
cds_ip = get_cds_implied_price(
    cds_name=cds_name,
    dates=cds_spread.index[:n],
    spreads=cds_spread['last_price'][:n]
)

In [ ]:
res = {}
for i, (date, row) in enumerate(cds_spread.iterrows()):
    rfc = risk_free_curve.query(f'date == "{date:{dt_fmt}}"').set_index('tenor')['last_price']
    cds_curve_ = cds_curve.loc[cds_curve['date'] == date].set_index('tenor')['last_price']
    spread = row['last_price']

    cds = get_isda_cds(
        cds_specs=cds_specs,
        risk_free_curve=rfc,
        trade_date=date,
        cds_spread=spread,
        notional=notional,
        coupon=coupon
    )
    res[date] = {'npv': cds.NPV(), 'cds_implied_price': 100 * (1+cds.fairUpfront()), 'spread': spread}
    
    if n and i + 1 == n:
        break

    # print(f"{date:%Y-%m-%d} NPV: {cds.NPV()}, Price: {100 * (1+cds.fairUpfront())}, spread: {spread}")
    # break

In [ ]:
res = pd.DataFrame(res).T
res

In [ ]:
pe = res['cds_implied_price'].rename('gio_pricer')
be = cds_ip.set_index('date')['cds_implied_price'].rename('bbg_pricer')
diff = (pe - be).rename('diff')

In [ ]:
pd.concat([pe, be, diff], axis=1)